# 데이터 준비하기: 다듬기, 변형, 병합

## 데이터 합치기

- pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다.
    - SQL이나 다른 관계형 데이터 페이스의 join과 유사하다.
- pandas.concat은 하나의 축을 따라 객체를 이어붙인다.
- combine_first 인스턴스 메서드는 구 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

## 데이터베이스 스타일로 DataFrame 합치기

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [3]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


,data2,key
0,0,a
1,1,b
2,2,d


- 일 대 다의 예제다.
- df1 데이터는 key 칼럼에 여러 개의 a, b를 가지고 있고 df2의 key 칼럼은 유일한 로우를 가지고 있다.

In [6]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 어떤 칼럼을 병합할 것인지 명시하지 않았다.
- merge 함수는 겹치는 칼럼의 이름을 키로 사용한다.
- 키를 명시적으로 지정하는 습관을 들이는 것이 좋다.
- merge 함수는 기본적으로 inner join을 수행하여 교집합인 결과를 반환한다.

In [7]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 두 객체에 공통되는 칼럼 이름이 하나도 없다면 따로 지정할 수 있다.

In [8]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [9]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [10]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


* left_on, right_on 키워드를 이용해서 칼럼을 지정했다.

- merge 함수는 기본적으로 inner join을 수행하여 교집합인 결과를 반환한다.
- 그렇기 때문에 c, d 에 해당하는 값은 제외되었다.
- how 인자로 left, right, outer를 넘겨서 각각 왼쪽 우선 외부조인, 오른쪽 우선 외부조인, 완전 외부 조인을 수행할 수 있다.
- 완전 외부조인은 합집합인 결과를 반환한다.
- 왼쪽 우선 외부조인과 오른쪽 우선 외부조인은 각각 왼쪽 혹은 오른쪽의 모든 로우를 포함하는 결과를 반환한다.

In [11]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [12]:
pd.merge(df1, df2, how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [13]:
pd.merge(df1, df2, how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


In [14]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


* 다 대 일 병합의 inner, left, right, outer는 직관적이어서 금방 이해할 수 있다.

- 다 대 다 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다.

In [15]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})

In [16]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

In [17]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [18]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [19]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [20]:
pd.merge(df1, df2, on='key', how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,5.0,b,1
3,0.0,b,3
4,1.0,b,3
5,5.0,b,3
6,2.0,a,0
7,4.0,a,0
8,2.0,a,2
9,4.0,a,2


,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,5.0,b,1
3,0.0,b,3
4,1.0,b,3
5,5.0,b,3
6,2.0,a,0
7,4.0,a,0
8,2.0,a,2
9,4.0,a,2


- 다 대 다 조인은 두 로우의 데카르트 곱을 반환한다.
- df1에는 b 로우가 3개, df2에는 b 로우가 2개, 결과는 6개의 b 로우가 된다.

In [21]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


* 여러 개의 키를 병합하려면 칼럼 이름이 들어간 리스트를 넘기면 된다.

In [22]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

In [23]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

In [24]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


* 여러 개의 키가 들어있는 튜플의 배열이 단일 조인 키로 사용된다고 생각하자.
* 칼럼과 칼럼을 조인할 대 전달한 DataFrame 객체의 색인은 무시된다.

* 머지 연산에서 고려해야 할 마지막 사항은 겹치는 칼럼 이름에 대한 처리다.
* 축의 이름을 변경해서 수동으로 칼럼 이름을 겹치게 할 수도 있고,
* merge 함수에 있는 suffixes 인자를 통해 두 DataFrame 객체에서 겹치는 칼럼 이름 뒤에 붙일 문자열을 지정할 수도 있다.

In [25]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [26]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### merge 함수 인자
* how: 조인 방법, inner, outer, left, right
* on: 조인하려는 로우 이름, 반드시 두 DataFrame 객체 모두에 있는 이름이어야한다.
    * 만역 명시되지 않고 다른 조인키도 주어지지 않으면 left와 right에서 공통되는 칼럼은 조인키로 사용한다.
* left_on: 조인 키로 사용할 left DataFrame의 칼럼
* right_on: 조인 키로 사용할 right DataFrame의 칼럼
* left_index: 조인 키로 사용할 left DataFrame의 색인 로우
* right_index: 조인 키로 사용할 right DataFrmae의 색인 로우
* sort: 조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본값은 True. 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수 있다.
* suffixes: 칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플.
    * 기본값은 `('_x', '_y')`
* copy: False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다.
    * 기본 값은 항상 복사가 이루어진다.

## 색인 머지하기

- 머지하려는 키가 DataFrame의 색인일 수 있다.
- left_index=True, right_index=True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.

In [27]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                   'value': range(6)})

In [28]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [29]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [30]:
right1

,group_val
a,3.5
b,7.0


,group_val
a,3.5
b,7.0


In [31]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


- left의 key 컬럼과 right의 index 컬럼을 기준으로 결합했다.

In [32]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


* 계층 색인된 데이터는 복잡하다.
* 계층 색인은 리스트로 여러 개의 칼럼을 지정해서 머지해야 한다.

In [33]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5)})

In [34]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])

In [35]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [36]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [37]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


In [38]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


* 양쪽에 공통으로 있는 여러 개의 색인을 머지할 수도 있다.

# 중간생략

## 문자열 다루기

- pandas는 배열 데이터 전체에 쉽게 정규표현식을 적용하고 추가적으로 누락된 데이터를 편리하게 처리할 수 있는 기능을 포햠하고 있다.

## 문자열 객체 메서드

- 문자열을 다뤄야 하는 대부분의 프로그램은 내장 문자열 메서드만으로 충분하다.

In [39]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

['a', 'b', '  guido']

- split 메서드는 종종 공백문자를 제거하는 strip 메서드와 조합해서 사용하기도 한다.

In [40]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

['a', 'b', 'guido']

- 이렇게 분리된 문자열은 더하기 연산을 통해 '::' 문자열과 합칠 수도 있다.

In [41]:
first, second, third = pieces

In [42]:
first +  '::' + second + '::' + third

'a::b::guido'

'a::b::guido'

- 좀 더 빠르고 파이썬스러운 방법은 join을 사용하는 것이다.

In [43]:
'::'.join(pieces)

'a::b::guido'

'a::b::guido'

- 일치하는 부분 문자열의 위치를 찾는 방법
- index, find, in

In [44]:
'guido' in val

True

True

In [45]:
val.index(',')

1

1

In [46]:
# val.index(':') # ValueError

In [47]:
val.find(',')

1

1

In [48]:
val.find(':')

-1

-1

- find와 index의 차이점은 index의 경우 문자열을 찾지 못하면 예외를 발생시킨다는 점이다.
- find는 문자열을 찾지 못하면 -1을 반환한다.

- count는 특정 부분 문자열이 몇 건 발견되었는지를 반환

In [49]:
val.count(',')

2

2

- replace는 찾아낸 패턴을 다른 문자열로 치환한다.
- 패턴을 삭제하기 위한 방법으로 자주 사용한다.

In [50]:
val.replace(',', '::')

'a::b::  guido'

'a::b::  guido'

In [51]:
val.replace(',', '')

'ab  guido'

'ab  guido'

## 정규표현식

- re 모듈 함수는 패턴 매칭, 치환, 분리의 세 가지로 나눌 수 있다.
- 정규표현식은 텍스트 안에 존재하는 패턴을 표현하고 이를 여러 가지 다양한 목적으로 사용할 수 있도록 되어 있다.

- 여러 가지 공백문자(탭, 스페이스, 개행문자)가 포함된 문자열을 나누고 싶다면 하나 이상의 공백문자를 의미하는 `\s+`를 사용해서 문자열을 분리할 수 있다.

In [52]:
import re

In [53]:
text = "foo     bar\t baz    \tqux"

In [54]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

['foo', 'bar', 'baz', 'qux']

- 먼저 정규표현식ㅇ이 컴파일되고 그 다음에 split 메서드가 실행된다.
- re.compile을 통해 직접 정규표현식을 컴파일하고 그렇게 얻은 정규표현식 객체를 재사용하는 것도 가능하다.

In [55]:
regex = re.compile('\s+')

In [56]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

['foo', 'bar', 'baz', 'qux']

- 정규표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드를 사용한다.

In [57]:
regex.findall(text)

['     ', '\t ', '    \t']

['     ', '\t ', '    \t']

- 정규표현식 안에서 `\`문자가 이스케이프되는 것을 피하려면 raw 문자열 표기법으로 문제를 회피할 수 있다.
- `r''`

- 같은 정규표현식을 다른 문자열에도 적용해야 한다면 re.compile을 이용해서 정규표현식 객체를 만들어 사용하는 방법을 추천한다.

- match와 search는 findall 메서드와 관련이 있다.
- findall은 문자열에서 일치하는 모든 부분의 문자열을 찾아주지만 search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다.

In [58]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [59]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [60]:
regex = re.compile(pattern, flags=re.IGNORECASE)

- findall 메서드를 사용하면 주어진 텍스트의 모든 이메일 주소를 확인할 수 있다.

In [61]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

- search는 텍스트에서 첫 번째 이메일 주소만 확인할 수 있다.
- 이 정규표현식에 대한 match 객체는 그 패턴이 문자열 안에서 위치하는 시작점과 끝점만을 알려준다.

In [62]:
m = regex.search(text)

In [63]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [64]:
text[m.start():m.end()]

'dave@google.com'

'dave@google.com'

In [65]:
print(regex.match(text)) # 이건 무슨 의미지?

None
None


- sub 메서드는 찾을 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.

In [66]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



- 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사의 세 가지 컴포넌트로 나누어야 한다면 각 패컨을 괄호로 묶어준다.

In [67]:
pattern = r'([A-Z0-9._%+-]+)@([[A-Z0-9.-]+)\.([A-Z]{2,4})'

In [68]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [69]:
m = regex.match('wesm@bright.net')

In [70]:
m.groups()

('wesm', 'bright', 'net')

('wesm', 'bright', 'net')

- 패턴에 그룹이 있다면 findall 메서드는 튜플의 리스트를 반환한다.

In [71]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

- sub 메서드는 `\1, \2` 같은 특수한 기호룰 사용해서 각 패턴의 그룹에 접근할 수 있다.

In [72]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



- 정규표현식에서 변수명을 줄 수도 있다.

In [73]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [74]:
m = regex.match('wesm@bright.net')

In [75]:
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

###  정규표현식 메서드
- findall: 문자열에서 겹치지 않는, 발견된 모든 패턴을 반환한다.
- finditer: 이터레이터를 통해 하나씩 반환한다.
- match: 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다(?)
    - 일치하는 패턴이 있다면 match 객체를 반환, 그렇지 않을 경우 None을 반환

## pandas의 벡터화된 문자열 함수

- 문자열과 정규표현식 메서드는 data.map을 사용해서 각 값에 적용할 수 있지만 NA 값을 만나면 실패한다.
- 이런 문제에 대처하기 위해 Series에는 NA 값을 건너뛰게 하는 간결한 문자열 처리 메서드가 있다.
- 예들 들어 각 이메일 주소가 'gmail'을 포함하고 있는지 str.contains를 이용해 검사할 수 있다.

In [76]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [77]:
data = Series(data)

In [78]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [79]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [80]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

- IGNORECASE 같은 re 옵션과 함께 정규표현식도 사용할 수 있다.

In [81]:
import re
pattern = r'[A-Z0-9._%+-]+@A-Z0-9.-]+\.[A-Z]{2,4}'

In [82]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave      []
Rob       []
Steve     []
Wes      NaN
dtype: object

Dave      []
Rob       []
Steve     []
Wes      NaN
dtype: object